# card_stack

> Card stack UI operations — navigation, viewport, mode switching, and response builders

In [ ]:
#| default_exp routes.card_stack

In [ ]:
#| export
from typing import List, Dict, Any, Tuple, Callable

from fasthtml.common import APIRouter

from cjm_fasthtml_card_stack.core.models import CardStackState, CardStackUrls
from cjm_fasthtml_card_stack.core.constants import DEFAULT_CARD_WIDTH
from cjm_fasthtml_card_stack.routes.handlers import (
    build_slots_response, build_nav_response,
    card_stack_navigate, card_stack_update_viewport, card_stack_save_width,
)

from cjm_fasthtml_interactions.core.state_store import get_session_id

from cjm_transcript_segmentation.components.card_stack_config import (
    SEG_CS_CONFIG, SEG_CS_IDS,
)
from cjm_transcript_segmentation.models import SegmentationUrls
from cjm_transcript_segmentation.components.segment_card import (
    create_segment_card_renderer
)
from cjm_transcript_segmentation.routes.core import (
    WorkflowStateStore, _to_segments, _load_seg_context, _get_seg_state,
    _build_card_stack_state, _update_seg_state,
)

## Response Builders

Shared helpers that assemble OOB response tuples for card stack operations.

- **Slots only** — early returns and exit-split-mode (just viewport section swaps)
- **Navigation response** — navigation and enter-split-mode (slots + progress + focus)

In [ ]:
#| export
def _make_renderer(
    urls: SegmentationUrls,  # URL bundle
    is_split_mode: bool = False,  # Whether split mode is active
    caret_position: int = 0,  # Caret position for split mode
) -> Any:  # Card renderer callback
    """Create a segment card renderer with captured URLs and mode state."""
    return create_segment_card_renderer(
        split_url=urls.split,
        merge_url=urls.merge,
        enter_split_url=urls.enter_split,
        exit_split_url=urls.exit_split,
        is_split_mode=is_split_mode,
        caret_position=caret_position,
    )

In [ ]:
#| export
def _build_slots_oob(
    segment_dicts: List[Dict[str, Any]],  # Serialized segments
    state: CardStackState,  # Card stack viewport state
    urls: SegmentationUrls,  # URL bundle
    caret_position: int = 0,  # Caret position for split mode
) -> List[Any]:  # OOB slot elements
    """Build OOB slot updates for the viewport sections."""
    is_split = state.active_mode == "split"
    return build_slots_response(
        card_items=_to_segments(segment_dicts),
        state=state,
        config=SEG_CS_CONFIG,
        ids=SEG_CS_IDS,
        urls=urls.card_stack,
        render_card=_make_renderer(urls, is_split_mode=is_split, caret_position=caret_position),
    )

In [ ]:
#| export
def _build_nav_response(
    segment_dicts: List[Dict[str, Any]],  # Serialized segments
    state: CardStackState,  # Card stack viewport state
    urls: SegmentationUrls,  # URL bundle
    caret_position: int = 0,  # Caret position for split mode
) -> Tuple:  # OOB elements (slots + progress + focus)
    """Build OOB response for navigation and mode changes."""
    is_split = state.active_mode == "split"
    return build_nav_response(
        card_items=_to_segments(segment_dicts),
        state=state,
        config=SEG_CS_CONFIG,
        ids=SEG_CS_IDS,
        urls=urls.card_stack,
        render_card=_make_renderer(urls, is_split_mode=is_split, caret_position=caret_position),
        progress_label="Segment",
    )

## Navigation Handler

In [ ]:
#| export
def _handle_seg_navigate(
    state_store: WorkflowStateStore,  # The workflow state store
    workflow_id: str,  # The workflow identifier
    sess,  # FastHTML session object
    direction: str,  # Navigation direction: "up", "down", "first", "last", "page_up", "page_down"
    urls: SegmentationUrls,  # URL bundle for segmentation routes
):  # OOB slot updates with progress and focus
    """Navigate to a different segment in the viewport using OOB slot swaps."""
    session_id = get_session_id(sess)
    ctx = _load_seg_context(state_store, workflow_id, session_id)
    segments = _to_segments(ctx.segment_dicts)
    
    state = _build_card_stack_state(ctx)
    renderer = _make_renderer(urls)
    
    result = card_stack_navigate(
        direction=direction,
        card_items=segments,
        state=state,
        config=SEG_CS_CONFIG,
        ids=SEG_CS_IDS,
        urls=urls.card_stack,
        render_card=renderer,
        progress_label="Segment",
    )
    
    _update_seg_state(state_store, workflow_id, session_id, focused_index=state.focused_index)
    return result

## Enter/Exit Split Mode Handlers

In [ ]:
#| export
def _handle_seg_enter_split_mode(
    state_store: WorkflowStateStore,  # The workflow state store
    workflow_id: str,  # The workflow identifier
    request,  # FastHTML request object
    sess,  # FastHTML session object
    segment_index: int,  # Index of segment to enter split mode for
    urls: SegmentationUrls,  # URL bundle for segmentation routes
):  # OOB slot updates with split mode active for focused segment
    """Enter split mode for a specific segment."""
    session_id = get_session_id(sess)
    ctx = _load_seg_context(state_store, workflow_id, session_id)
    _update_seg_state(state_store, workflow_id, session_id, focused_index=segment_index)

    state = _build_card_stack_state(ctx, active_mode="split")
    state.focused_index = segment_index
    return _build_nav_response(ctx.segment_dicts, state, urls, caret_position=0)

In [ ]:
#| export
def _handle_seg_exit_split_mode(
    state_store: WorkflowStateStore,  # The workflow state store
    workflow_id: str,  # The workflow identifier
    request,  # FastHTML request object
    sess,  # FastHTML session object
    urls: SegmentationUrls,  # URL bundle for segmentation routes
):  # OOB slot updates with split mode deactivated
    """Exit split mode."""
    session_id = get_session_id(sess)
    ctx = _load_seg_context(state_store, workflow_id, session_id)
    state = _build_card_stack_state(ctx)
    return _build_slots_oob(ctx.segment_dicts, state, urls)

## Update Viewport Handler

Handler for updating the viewport when card count changes. Does a full viewport swap (outerHTML) since the number of slots changes.

In [ ]:
#| export
async def _handle_seg_update_viewport(
    state_store: WorkflowStateStore,  # The workflow state store
    workflow_id: str,  # The workflow identifier
    request,  # FastHTML request object
    sess,  # FastHTML session object
    visible_count: int,  # New number of visible cards
    urls: SegmentationUrls,  # URL bundle for segmentation routes
):  # Full viewport component (outerHTML swap)
    """Update the viewport with a new card count.

    Does a full viewport swap because the number of slots changes.
    Saves the new visible_count and is_auto_mode to state.
    """
    session_id = get_session_id(sess)
    ctx = _load_seg_context(state_store, workflow_id, session_id)
    segments = _to_segments(ctx.segment_dicts)
    
    state = _build_card_stack_state(ctx)
    renderer = _make_renderer(urls)
    
    result = card_stack_update_viewport(
        visible_count=visible_count,
        card_items=segments,
        state=state,
        config=SEG_CS_CONFIG,
        ids=SEG_CS_IDS,
        urls=urls.card_stack,
        render_card=renderer,
    )
    
    # Read is_auto from form data (passed by client JS)
    form_data = await request.form()
    is_auto_str = form_data.get("is_auto", "false")
    is_auto_mode = is_auto_str.lower() == "true"
    
    _update_seg_state(
        state_store, workflow_id, session_id,
        visible_count=state.visible_count,
        is_auto_mode=is_auto_mode,
    )
    return result

## Save Width Handler

Saves the card stack width to server state for persistence across page loads.

In [ ]:
#| export
def _handle_seg_save_width(
    state_store: WorkflowStateStore,  # The workflow state store
    workflow_id: str,  # The workflow identifier
    sess,  # FastHTML session object
    card_width: int,  # Card stack width in rem
) -> None:  # No response body (swap=none on client)
    """Save the card stack width to server state.
    
    Called via debounced HTMX POST from the width slider.
    Returns nothing since the client uses hx-swap='none'.
    """
    session_id = get_session_id(sess)
    seg_state = _get_seg_state(state_store, workflow_id, session_id)
    current_width = seg_state.get("card_width", DEFAULT_CARD_WIDTH)
    state = CardStackState(card_width=current_width)
    card_stack_save_width(state, card_width, SEG_CS_CONFIG)
    _update_seg_state(state_store, workflow_id, session_id, card_width=state.card_width)

## Router Initialization

Creates the card stack router with navigation, viewport, and split mode routes.

In [ ]:
#| export
def init_card_stack_router(
    state_store: WorkflowStateStore,  # The workflow state store
    workflow_id: str,  # The workflow identifier
    prefix: str,  # Route prefix (e.g., "/workflow/seg/card_stack")
    urls: SegmentationUrls,  # URL bundle (populated after routes defined)
) -> Tuple[APIRouter, Dict[str, Callable]]:  # (router, route_dict)
    """Initialize card stack routes for segmentation."""
    router = APIRouter(prefix=prefix)

    # -------------------------------------------------------------------------
    # Navigation
    # -------------------------------------------------------------------------

    @router
    def nav_up(request, sess):
        """Navigate to previous segment."""
        return _handle_seg_navigate(state_store, workflow_id, sess, direction="up", urls=urls)

    @router
    def nav_down(request, sess):
        """Navigate to next segment."""
        return _handle_seg_navigate(state_store, workflow_id, sess, direction="down", urls=urls)

    @router
    def nav_first(request, sess):
        """Navigate to first segment."""
        return _handle_seg_navigate(state_store, workflow_id, sess, direction="first", urls=urls)

    @router
    def nav_last(request, sess):
        """Navigate to last segment."""
        return _handle_seg_navigate(state_store, workflow_id, sess, direction="last", urls=urls)

    @router
    def nav_page_up(request, sess):
        """Navigate up by page."""
        return _handle_seg_navigate(state_store, workflow_id, sess, direction="page_up", urls=urls)

    @router
    def nav_page_down(request, sess):
        """Navigate down by page."""
        return _handle_seg_navigate(state_store, workflow_id, sess, direction="page_down", urls=urls)

    # -------------------------------------------------------------------------
    # Viewport and Width
    # -------------------------------------------------------------------------

    @router
    async def update_viewport(request, sess, visible_count: int):
        """Update viewport with new card count (full outerHTML swap)."""
        return await _handle_seg_update_viewport(
            state_store, workflow_id, request, sess, visible_count, urls=urls,
        )

    @router
    def save_width(request, sess, card_width: int):
        """Save card stack width to server state."""
        return _handle_seg_save_width(state_store, workflow_id, sess, card_width)

    # -------------------------------------------------------------------------
    # Split Mode
    # -------------------------------------------------------------------------

    @router
    def enter_split(request, sess, segment_index: int):
        """Enter split mode for a specific segment."""
        return _handle_seg_enter_split_mode(
            state_store, workflow_id, request, sess, segment_index, urls=urls,
        )

    @router
    def exit_split(request, sess):
        """Exit split mode."""
        return _handle_seg_exit_split_mode(state_store, workflow_id, request, sess, urls=urls)

    # -------------------------------------------------------------------------
    # Route Dict
    # -------------------------------------------------------------------------

    routes = {
        "nav_up": nav_up,
        "nav_down": nav_down,
        "nav_first": nav_first,
        "nav_last": nav_last,
        "nav_page_up": nav_page_up,
        "nav_page_down": nav_page_down,
        "update_viewport": update_viewport,
        "save_width": save_width,
        "enter_split": enter_split,
        "exit_split": exit_split,
    }

    return router, routes

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()